# Generating Predictions for all Datasets in the Benchmarking Data

In [3]:
import pandas as pd
import matplotlib.pyplot as plt

import sys
sys.path.append('../')
import conf #This is the local config file for setting the supplements directory to your local environment

odir = conf.SUPPLEMENTS_DIR+'Benchmarking (Fig3-4)/'

#load benchmarking data
y_data = pd.read_csv(odir+'benchmark_dataset_Y.tsv', sep = '\t', index_col = 0)
st_data = pd.read_csv(odir + 'benchmark_dataset_Y.tsv', sep = '\t', index_col = 0)


#load Y meta data
y_meta = pd.read_csv('meta_y.tsv', sep = '\t')
#expand meta so that each row corresponds to a single kinase condition
y_meta['Kinases'] = y_meta['Kinases'].apply(lambda x: x.split(','))
y_meta = y_meta.explode('Kinases')
y_meta.rename({'Unnamed: 0': 'Dataset'}, axis = 1, inplace = True)
y_meta = y_meta.drop_duplicates()
y_meta.reset_index(inplace = True)
#load st meta data
st_meta = pd.read_csv('meta_st.tsv', sep = '\t')
#expand meta so that each row corresponds to a single kinase condition
st_meta['Kinases'] = st_meta['Kinases'].apply(lambda x: str(x).split(','))
st_meta = st_meta.explode('Kinases')
st_meta.rename({'Unnamed: 0': 'Dataset'}, axis = 1, inplace = True)
st_meta = st_meta.drop_duplicates()
st_meta.reset_index(inplace = True)

#Load mapped PhosphoSitePlus
PSP = pd.read_csv(odir + '/psp_mapped_Nov2021.tsv', sep = '\t', index_col = 0)

## KSTAR

For using KSTAR with benchmarking data, we looked for enriched kinases either in the upregulated (FC > 1) or downregulated sites (FC < 1) (depending on the direction of perturbation). To do that, needed to split the benchmarking dataset between the up and downregulated conditions.

In [ ]:
#need to split the dataset between up and down
up_datasets = y_meta.loc[y_meta['Direction'] == 'Up','Dataset'].unique()
down_datasets = y_meta.loc[y_meta['Direction'] == 'Down','Dataset'].unique()
#run for upregulated sites
y_up = y_data[['KSTAR_ACCESSION','KSTAR_SITE','KSTAR_PEPTIDE','KSTAR_NUM_COMPENDIA','KSTAR_NUM_COMPENDIA_CLASS']+list(up_datasets)]
y_up.to_csv('benchmark_dataset_Y_up.tsv',sep='\t')
y_down = y_data[['KSTAR_ACCESSION','KSTAR_SITE','KSTAR_PEPTIDE','KSTAR_NUM_COMPENDIA','KSTAR_NUM_COMPENDIA_CLASS']+list(down_datasets)]
y_down.to_csv('benchmark_dataset_Y_down.tsv',sep='\t')

In [ ]:
#split st (currently not an up datasets)
up_datasets = st_meta.loc[st_meta['Direction'] == 'Up','Dataset'].unique()
down_datasets = st_meta.loc[st_meta['Direction'] == 'Down','Dataset'].unique()
#run for upregulated sites
st_up = st_data[['KSTAR_ACCESSION','KSTAR_SITE','KSTAR_PEPTIDE','KSTAR_NUM_COMPENDIA','KSTAR_NUM_COMPENDIA_CLASS']+list(up_datasets)]
#st_up.to_csv('benchmark_dataset_ST_up.tsv',sep='\t')
st_down = st_data[['KSTAR_ACCESSION','KSTAR_SITE','KSTAR_PEPTIDE','KSTAR_NUM_COMPENDIA','KSTAR_NUM_COMPENDIA_CLASS']+list(down_datasets)]
#st_down.to_csv('benchmark_dataset_ST_down.tsv',sep='\t')

The above data was then run through the nextflow implementation of KSTAR, as these benchmarking datasets (particularly for ST kinases) were large and required high performance runs. For more details on this implementation, visit the KSTAR documentation.

## KSEA

In [ ]:
from Algorithms import KSEA      #custom made package based on original paper by Casado et al.

In [ ]:
ksea_y = KSEA.KSEA(y_data, PSP,kinase_col = 'GENE', log_transform = True)
ksea_y.runKSEA()
ksea_st = KSEA.KSEA(st_data, PSP,kinase_col = 'GENE', log_transform = True)
ksea_st.runKSEA()
ksea = {'Y':ksea_y, 'ST':ksea_st}

In [ ]:
ksea['Y'].saveResults(fname = 'benchmark_Y', odir = './KSEA_results/')
ksea['ST'].saveResults(fname = 'benchmark_ST', odir = './KSEA_results/')

## PTM-SEA

PTM-SEA predictions were generated using their provided R gui, but the data needed to be formatted as indicated by the authors:

In [ ]:
#Y
#take averages of duplicate sites (PTM-SEA requires this to be done beforehand)
for_ptm_sea = y_data.groupby(['KSTAR_ACCESSION','KSTAR_SITE','KSTAR_PEPTIDE','KSTAR_NUM_COMPENDIA','KSTAR_NUM_COMPENDIA_CLASS']).mean().reset_index()
#generate PTM-SEA formatted ids
for_ptm_sea['KSTAR_SUBSTRATE'] = for_ptm_sea['KSTAR_ACCESSION']+'_'+for_ptm_sea['KSTAR_SITE']
for_ptm_sea['id.uniprot'] = for_ptm_sea['KSTAR_SUBSTRATE'].apply(lambda x: x.split('_')[0]+';'+x.split('_')[1]+'-p')
for_ptm_sea['id.flanking'] = for_ptm_sea['KSTAR_PEPTIDE'].apply(lambda x: x.upper()+'-p')
for_ptm_sea.drop_duplicates(subset = ['id.uniprot'], inplace = True)
#for_ptm_sea.to_csv('for_ptm_sea_Y.tsv', sep ='\t')

In [ ]:
#ST
#take averages of duplicate sites (PTM-SEA requires this to be done beforehand)
for_ptm_sea = st_data.groupby(['KSTAR_ACCESSION','KSTAR_SITE','KSTAR_PEPTIDE','KSTAR_NUM_COMPENDIA','KSTAR_NUM_COMPENDIA_CLASS']).mean().reset_index()
#generate PTM-SEA formatted ids
for_ptm_sea['KSTAR_SUBSTRATE'] = for_ptm_sea['KSTAR_ACCESSION']+'_'+for_ptm_sea['KSTAR_SITE']
for_ptm_sea['id.uniprot'] = for_ptm_sea['KSTAR_SUBSTRATE'].apply(lambda x: x.split('_')[0]+';'+x.split('_')[1]+'-p')
for_ptm_sea['id.flanking'] = for_ptm_sea['KSTAR_PEPTIDE'].apply(lambda x: x.upper()+'-p')
for_ptm_sea.drop_duplicates(subset = ['id.uniprot'], inplace = True)
#for_ptm_sea.to_csv('for_ptm_sea_ST.tsv', sep ='\t')

Run data through the gui, then process to extract only kinase signatures
- y_score: enrichment scores obtained for tyrosine kinase datasets
- y_pvals: significance of scores obtained for tyrosine kinase datasets
- st_score: enrichment scores obtained for serine/threonine kinase datasets
- st_pvals: significance of scores obtained for serine/threonine kinase datasets

In [ ]:
#identify the kinase signatures in results to keep
keep = []
new_index = {}
for pert in y_score.index:
    if 'KINASE' in pert:
        keep.append(pert)
        if '/' in pert: 
            new_index[pert] = pert.split('/')[-1]
        else:
            new_index[pert] = pert.split('_')[-1]
#keep only the kinase signatures, and simplify the naming to match other algorithms
y_score = y_score.loc[keep]
y_score.rename(new_index, inplace = True)
y_pval = y_pval.loc[keep]
y_pval.rename(new_index, inplace = True)

#replace colnames from 'data.' to 'data:'
new_col = {}
for col in y_score.columns:
    new_col[col] = col.replace('.',':')
    
y_score.rename(new_col, axis = 1, inplace = True)
y_pval.rename(new_col, axis = 1, inplace = True)

In [ ]:
#identify the kinase signatures in results to keep
keep = []
new_index = {}
for pert in st_score.index:
    if 'KINASE' in pert:
        keep.append(pert)
        if '/' in pert: 
            new_index[pert] = pert.split('/')[-1]
        else:
            new_index[pert] = pert.split('_')[-1]
#keep only the kinase signatures, and simplify the naming to match other algorithms
st_score = st_score.loc[keep]
st_score.rename(new_index, inplace = True)
st_pval = st_pval.loc[keep]
st_pval.rename(new_index, inplace = True)

#replace colnames from 'data.' to 'data:'
new_col = {}
for col in st_score.columns:
    new_col[col] = col.replace('data.','data:')
    if col == 'data.AKT_MK.2206_Wiechmann':
        new_col[col] = 'data:AKT_MK-2206_Wiechmann'
    elif col == 'data.AURK_MLN8054_0.25uM_Kellenbach':
        new_col[col] = 'data:AURK_MLN8054_0.25uM_Kellenbach'
    elif col ==  'data.CDK1_inhib_RO.3306_Petrone':
        new_col[col] = 'data:CDK1_inhib_RO-3306_Petrone'
    
st_score.rename(new_col, axis = 1, inplace = True)
st_pval.rename(new_col, axis = 1, inplace = True)

Saved data in FigShare is the reformatted results obtained via the above approach

## KARP

In [ ]:
from Algorithms import KARP    #custom made package based on original KARP paper

In [ ]:
#Y: generate control datasets (FC = 1 for all identified sites)
data_cols = [col for col in y_data.columns if 'data:' in col]
y_control = y_data.copy()
for col in data_cols:
    change = ~y_control[col].isna().values
    y_control.loc[change, col] = 1

#ST: generate control datasets (FC = 1 for all identified sites)
data_cols = [col for col in st_data.columns if 'data:' in col]
st_control = st_data.copy()
for col in data_cols:
    change = ~st_control[col].isna().values
    st_control.loc[change, col] = 1

In [ ]:
#Y: Run KARP on both control and condition dataset, then take the difference in K-score
karp_y = KARP.KARP(y_data, PSP, kinase_col = 'GENE')
karp_y.runKARP()
karp_control_y = KARP.KARP(y_control, PSP, kinase_col = 'GENE')
karp_control_y.runKARP()
y_res = karp_y.K - karp_control_y.K
#ST: Run KARP on both control and condition dataset, then take the difference in K-score
karp_st = KARP.KARP(st_data, PSP, kinase_col = 'GENE')
karp_st.runKARP()
karp_control_st = KARP.KARP(st_control, PSP, kinase_col = 'GENE')
karp_control_st.runKARP()
st_res = karp_st.K - karp_control_st.K
karp = {'Y':y_res, 'ST':st_res}

## KEA3

In [ ]:
from Algorithms import KEA3      #custom built package for using the KEA3 web app with mapped datasets

In [ ]:
#Y
#need to split the dataset between up and downregulated
up_datasets = y_meta.loc[y_meta['Direction'] == 'Up','Dataset'].unique()
down_datasets = y_meta.loc[y_meta['Direction'] == 'Down','Dataset'].unique()
#run for upregulated sites
kea_y_up = KEA3.runKEA3onDataset(y_data[['KSTAR_ACCESSION','KSTAR_SITE','KSTAR_PEPTIDE','KSTAR_NUM_COMPENDIA','KSTAR_NUM_COMPENDIA_CLASS']+list(up_datasets)], direction = 'Up')
#extract mean ranks in matrix format from provided KEA3 results
rearranged = pd.DataFrame(None, columns = kea_y_up['Query Name'].unique(), index = kea_y_up['TF'].unique())
for exp in kea_y_up['Query Name'].unique():
    tmp = kea_y_up[kea_y_up['Query Name'] == exp]
    tmp.index = tmp['TF']
    rearranged.loc[tmp.index, exp] = tmp['Score']

kea_y_up = rearranged.astype(float)


#run for downregulated sites
kea_y_down = KEA3.runKEA3onDataset(y_data[['KSTAR_ACCESSION','KSTAR_SITE','KSTAR_PEPTIDE','KSTAR_NUM_COMPENDIA','KSTAR_NUM_COMPENDIA_CLASS']+list(down_datasets)], direction = 'Down')
#extract mean ranks in matrix format from provided KEA3 results
rearranged = pd.DataFrame(None, columns = kea_y_down['Query Name'].unique(), index = kea_y_down['TF'].unique())
for exp in kea_y_down['Query Name'].unique():
    tmp = kea_y_down[kea_y_down['Query Name'] == exp]
    tmp.index = tmp['TF']
    rearranged.loc[tmp.index, exp] = tmp['Score']

kea_y_down = rearranged.astype(float)
#combine results
kea_y = pd.concat([kea_y_up, kea_y_down],axis = 1)

In [ ]:
#ST
#need to split the dataset between up and downregulated
up_datasets = st_meta.loc[st_meta['Direction'] == 'Up','Dataset'].unique()
down_datasets = st_meta.loc[st_meta['Direction'] == 'Down','Dataset'].unique()
#run for upregulated sites
kea_st_up = KEA3.runKEA3onDataset(st_data[['KSTAR_ACCESSION','KSTAR_SITE']+list(up_datasets)], direction = 'Up')
#extract mean ranks in matrix format from provided KEA3 results
rearranged = pd.DataFrame(None, columns = kea_st_up['Query Name'].unique(), index = kea_st_up['TF'].unique())
for exp in kea_st_up['Query Name'].unique():
    tmp = kea_st_up[kea_st_up['Query Name'] == exp]
    tmp.index = tmp['TF']
    rearranged.loc[tmp.index, exp] = tmp['Score']

kea_st_up = rearranged.astype(float)


#run for downregulated sites
kea_st_down = KEA3.runKEA3onDataset(st_data[['KSTAR_ACCESSION','KSTAR_SITE']+list(down_datasets)], direction = 'Down')
#extract mean ranks in matrix format from provided KEA3 results
rearranged = pd.DataFrame(None, columns = kea_st_down['Query Name'].unique(), index = kea_st_down['TF'].unique())
for exp in kea_st_down['Query Name'].unique():
    tmp = kea_st_down[kea_st_down['Query Name'] == exp]
    tmp.index = tmp['TF']
    rearranged.loc[tmp.index, exp] = tmp['Score']

kea_st_down = rearranged.astype(float)
#combine results
kea_st = pd.concat([kea_st_up, kea_st_down], axis = 1)